In [1]:
import pandas as pd

## Autour du paradoxe de Simpson
### Contexte
En 1972-1974, à Whickham, une ville du nord-est de l'Angleterre, située à environ 6,5 kilomètres au sud-ouest de Newcastle upon Tyne, un sondage d'un sixième des électeurs a été effectué afin d'éclairer des travaux sur les maladies thyroïdiennes et cardiaques (Tunbridge et al. 1977). Une suite de cette étude a été menée vingt ans plus tard (Vanderpump et al. 1995). Certains des résultats avaient trait au tabagisme et cherchaient à savoir si les individus étaient toujours en vie lors de la seconde étude. Par simplicité, nous nous restreindrons aux femmes et parmi celles-ci aux 1314 qui ont été catégorisées comme "fumant actuellement" ou "n'ayant jamais fumé". Il y avait relativement peu de femmes dans le sondage initial ayant fumé et ayant arrêté depuis (162) et très peu pour lesquelles l'information n'était pas disponible (18). La survie à 20 ans a été déterminée pour l'ensemble des femmes du premier sondage.
<br>
<br>
Lien du jeu de données : [jeu de données](https://gitlab.inria.fr/learninglab/mooc-rr/mooc-rr-ressources/blob/master/module3/Practical_session/Subject6_smoking.csv)

In [4]:
df = pd.read_csv("Subject6_smoking.csv")
df.head()

,Smoker,Status,Age
0,Yes,Alive,21.0
1,Yes,Alive,19.3
2,No,Dead,57.5
3,No,Alive,47.1
4,Yes,Alive,81.4


### Inspection des données

In [8]:
df.isna().any()

Smoker    False
Status    False
Age       False
dtype: bool

Comme nous le voyons il n'y a pas de données manquantes dans ce jeu de données

## Première tache
Représentez dans un tableau le nombre total de femmes vivantes et décédées sur la période en fonction de leur habitude de tabagisme. Calculez dans chaque groupe (fumeuses / non fumeuses) le taux de mortalité (le rapport entre le nombre de femmes décédées dans un groupe et le nombre total de femmes dans ce groupe)

In [37]:
smoker_agg =  df.groupby(['Smoker', 'Status']).agg('count')
smoker_agg.columns = ['Number_of_cases']
smoker_agg = smoker_agg.reset_index()
smoker_agg

,Smoker,Status,Number_of_cases
0,No,Alive,502
1,No,Dead,230
2,Yes,Alive,443
3,Yes,Dead,139


In [54]:
# Taux de mortalité dans le groupe de fumeuses
smoker_dead_num = float(smoker_agg[(smoker_agg.Smoker=="Yes") & (smoker_agg.Status=="Dead")]['Number_of_cases'])
mortality = 100*smoker_dead_num / smoker_agg[smoker_agg.Smoker=='Yes']['Number_of_cases'].sum()
print("Le taux de mortalité chez les fumeuses est de : "+str(round(mortality,2))+' %')


Le taux de mortalité chez les fumeuses est de : 23.88 %


In [55]:
# Taux de mortalité dans le groupe des non fumeuses
nosmoker_dead_num = float(smoker_agg[(smoker_agg.Smoker=="No") & (smoker_agg.Status=="Dead")]['Number_of_cases'])
mortality = 100*nosmoker_dead_num / smoker_agg[smoker_agg.Smoker=='No']['Number_of_cases'].sum()
print("Le taux de mortalité chez les fumeuses est de : "+str(round(mortality,2))+' %')


Le taux de mortalité chez les fumeuses est de : 31.42 %
